In [71]:
import requests, time, re, os, json
import pymysql, pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from bs4 import BeautifulSoup as bs
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials
with open('../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)
league = 'nfl'

In [2]:

today = date.today()
season = 2023
week = int(input("What week is it >>>>>> ? "))
if week < 10:
    strWeek = "0" + str(week)
else:
    strWeek = week

What week is it >>>>>> ?  1


In [3]:
# column structures for stat projections
projection_columns = ["outlet","date", "season", "week", "playerId", "name", "shortName", "pos", "team", 'GamesPlayed',
 'PassAttempts','PassCompletions','PassingYards', 'PassingYardsPerGame', 'TouchdownsPasses', 
 'InterceptionsThrown', 'PasserRating',
 'RushingAttempts','RushingYards', 'AverageYardsPerRush', 'RushingTouchdowns',
 'Targets', 'Receptions', 'ReceivingYards', 'YardsPerGame', 'AverageYardsPerReception','ReceivingTouchdowns',
 'FumblesLost',
 'FieldGoalsMade','FieldGoalAttempts','LongestFieldGoal','FieldGoals119Yards','FieldGoals119YardAttempts',
 'FieldGoals2029Yards','FieldGoals2029YardAttempts','FieldGoals3039Yards','FieldGoals3039YardAttempts',
 'FieldGoals4049Yards','FieldGoals4049YardAttempts','FieldGoals50Yards','FieldGoals50YardsAttempts',
 'ExtraPointsMade','ExtraPointsAttempted',
 'Interceptions','Safeties','Sacks','Tackles','DefensiveFumblesRecovered','ForcedFumbles','DefensiveTouchdowns', 
 'ReturnTouchdowns','PointsAllowed','PointsAllowedPerGame','NetPassingYardsAllowed','RushingYardsAllowed', 
 'TotalYardsAllowed', 'YardsAgainstPerGame', 'twoPt','FantasyPoints','FantasyPointsPerGame']

ranking_columns = ["outlet", "date", "season", "week", "group", "expert", "rank","name","playerId","team","pos", "high", "low"]

team_map = {
    "Jacksonville Jaguars":"JAX","Los Angeles Rams":"LA","Philadelphia Eagles":"PHI","Minnesota Vikings":"MIN",
    "Houston Texans":"HOU","Los Angeles Chargers":"LAC","Baltimore Ravens":"BAL","New England Patriots":"NE",
    "Carolina Panthers":"CAR","Denver Broncos":"DEN","Arizona Cardinals":"ARI","New Orleans Saints":"NO",
    "Detroit Lions":"DET","Pittsburgh Steelers":"PIT","Chicago Bears":"CHI","Seattle Seahawks":"SEA",
    "Buffalo Bills":"BUF","Tennessee Titans":"TEN","Atlanta Falcons":"ATL","Cincinnati Bengals":"CIN",
    "Kansas City Chiefs":"KC","Washington Redskins":"WAS","Dallas Cowboys":"DAL","Tampa Bay Buccaneers":"TB",
    "Green Bay Packers":"GB","New York Giants":"NYG","San Francisco 49ers":"SF","Cleveland Browns":"CLE",
    "Oakland Raiders":"OAK","Indianapolis Colts":"IND","Miami Dolphins":"MIA","New York Jets":"NYJ"}
team_mascot_map = {
    "Jaguars":"JAX","Rams":"LA","Eagles":"PHI","Vikings":"MIN",
    "Texans":"HOU","Chargers":"LAC","Ravens":"BAL","Patriots":"NE",
    "Panthers":"CAR","Broncos":"DEN","Cardinals":"ARI","Saints":"NO",
    "Lions":"DET","Steelers":"PIT","Bears":"CHI","Seahawks":"SEA",
    "Bills":"BUF","Titans":"TEN","Falcons":"ATL","Bengals":"CIN",
    "Chiefs":"KC","Redskins":"WAS","Cowboys":"DAL","Buccaneers":"TB",
    "Packers":"GB","Giants":"NYG","49ers":"SF","Browns":"CLE",
    "Raiders":"OAK","Colts":"IND","Dolphins":"MIA","Jets":"NYJ"}

In [4]:
print(season, week, strWeek)

2023 1 01


# ----------- CBS - PPR -----------

### cbs projections

In [9]:
url_projections = "https://www.cbssports.com/fantasy/football/stats/{pos}/{season}/{week}/projections/ppr/"
positions = ["QB", "RB", "WR", "TE", "K", "DST"]
tableClass = "TableBase-table"  
tableHeader = "TableBase-headTr"
headerClass = "Tablebase-tooltipInner"
tableRow = "TableBase-bodyTr"
tableD = "TableBase-bodyTd"

df_cbs_proj = pd.DataFrame(columns=projection_columns)

# loop through each position to retrieve HTML and convert to df
for p in range(len(positions)):
    
    time.sleep(3)
    #updating URL for each position
    url_formatted = url_projections.format(pos=positions[p], season=season, week=week)

    # retreiving HTML and converting it to soup
    r = requests.get(url_formatted)
    soup = bs(r.text)

    # accessing table with the data
    table = soup.find("table", class_= tableClass)

    
    # accounting for the difference in DEF headers
    if positions[p] == "DST":
        cols = ["pos", "team","name"]
    else:
        cols = ["playerId", "name", "shortName", "pos", "team"]
    
    ### grabbing column names from the thead for the position. These will be used to create the temp. pos dataframe
    #retrieving column names from the HTML
    for i in table.find_all("div", class_=headerClass):
        cols.append(''.join(filter(str.isalnum, i.text)))
        
    # accessing the data in the body
    body = table.find("tbody")
    # looping through rows
    data = []
    for tr in body.find_all("tr", class_=tableRow):
        # accounting for DST and populating pos as DST since it is not provided
        if positions[p] == "DST":
            player_data = ["DST"]
        else:
            player_data = []
        
        for td in tr.find_all("td", class_=tableD):
            
            if positions[p] == "DST":
            # pulling team name 
                span = td.find_all("span",class_="CellLogoNameLockup")
                if span:   
                    for s in span:
                        player_data.append(s.find("a")["href"].split("/")[3])
                        player_data.append(str.strip(td.text))
                
                # non-span <Td>, 
                else:
                    player_data.append(str.strip(td.text))
                    
            # processing table body for all pos except DST
            else:
                #the player name, id, pos, and team are all in spans. 
                #the spans are not present in the stat <td>'s
                span_short = td.find_all("span",class_="CellPlayerName--short")
                span_long = td.find_all("span",class_="CellPlayerName--long")

                # if the <td> has a span, the player info will be extracted
                if span_long:

                    for s in span_long:
                        # player Id from the href url
                        player_data.append(s.find("a")["href"].split("/")[4])
                        # player full name
                        player_data.append(str.strip(s.find("a").text).replace(".", ""))

                    for s in span_short:
                        # player short name
                        player_data.append(s.find("a").text.replace(".", ""))
                        #player position
                        player_data.append(str.strip(s.find("span", class_="CellPlayerName-position").text))
                        #player nfl team
                        player_data.append(str.strip(s.find("span", class_="CellPlayerName-team").text).replace("JAC", "JAX").replace("WAS", "WSH"))
            
                # non-span <Td>
                else:
                    player_data.append(str.strip(td.text))
        
        # creates the list of players, each player is a list with stats
        data.append(player_data)
    
    # converts list of list to data frame with the applicable columns pulled earlier
    pos_df = pd.DataFrame(data, columns=cols)
    
    # concats all of the position data to the master df
    df_cbs_proj = pd.concat([df_cbs_proj, pos_df], axis=0, ignore_index=True)

df_cbs_proj.loc[:,'outlet'] = "cbs"
df_cbs_proj.loc[:,'date'] = today
df_cbs_proj.loc[:,'season'] = season
df_cbs_proj.loc[:,'week'] = week
df_cbs_proj.loc[:,'LongestFieldGoal'] = np.nan

df_cbs_proj.shape

(433, 60)

In [10]:
df_cbs_proj.to_csv("../data/projection/weekly/cbs_proj_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

### cbs rankings

In [ ]:
url_rankings = "https://www.cbssports.com/fantasy/football/rankings/ppr/{pos}"#/weekly/"

positions = ["QB", "RB", "WR", "TE", "K", "DST", "FLEX"]

# key class names that will be targeted
parentDivClass = "rankings-table multi-authors hide-player-stats"  # contains all expert rankings (3 tables)
individualRankingDivClass = "experts-column triple"  # 3 of these for their 3 experts  
authorNameAClass = "author-name"
playersDivClass = "player-wrapper"  # the divs of interest are in here but it also includes data that is not needed 

df_cbs_ranking = pd.DataFrame(columns=ranking_columns)

for pos in positions:
    time.sleep(3)    
    # retreiving HTML and converting it to soup
    url_formatted = url_rankings.format(pos=pos)
    r = requests.get(url_formatted)
    soup = bs(r.text)

    # finding the tables with rankings
    rankingTables = soup.find_all("div", class_=individualRankingDivClass)
    
    # looping through the 3 expert ranks that are in their own tables
    player_ranking_data = []
    if pos == "FLEX":
        for rt in rankingTables:
            #extracting expert name
            expert = rt.find("a", class_=authorNameAClass).span.text

            #looping through the divs that contain all the player level ranking data
            ranks = rt.find("div", class_=playersDivClass)
            for p in ranks:
                temp = ["cbs", today, season, week, pos, expert]
                try:
                    temp.append(str.strip(p.find("div", class_="rank").text))  #expert rank, number  .text
                    temp.append(str.strip(p.find("span", class_="player-name").text).replace(".", ""))  #cbs shortName  .text
                    temp.append(str.strip(p.find("a")["href"].split("/")[4])) # cbs playerId is in the url
                    temp.append(str.strip(p.find("span", class_="team position").text.split()[0])) # contains the player nfl team 
                    temp.append(str.strip(p.find("span", class_="team position").text.split()[1])) # contains the player nfl position
                    temp.append(np.nan)
                    temp.append(np.nan)
                    player_ranking_data.append(temp)
                except:
                    continue
        
    elif pos == "DST":
        for rt in rankingTables:
            #extracting expert name
            expert = rt.find("a", class_=authorNameAClass).span.text

            #looping through the divs that contain all the player level ranking data
            ranks = rt.find("div", class_=playersDivClass)
            for p in ranks:
                temp = ["cbs", today, season, week, pos, expert]
                try:
                    team = str.strip(p.find("span", class_="player-name").text)
                    temp.append(str.strip(p.find("div", class_="rank").text))  #expert rank, number  .text
                    temp.append(team)  #cbs shortName  .text
                    temp.append(str.strip(p.find("a")["href"].split("/")[4])) # cbs playerId is in the url
                    temp.append(team) # contains the player nfl team 
                    temp.append(pos) # contains the player nfl position
                    temp.append(np.nan)
                    temp.append(np.nan)
                    player_ranking_data.append(temp)
                except:
                    continue
    
    else:
        for rt in rankingTables:
            #extracting expert name
            expert = rt.find("a", class_=authorNameAClass).span.text

            #looping through the divs that contain all the player level ranking data
            ranks = rt.find("div", class_=playersDivClass)
            for p in ranks:
                temp = ["cbs", today, season, week, pos, expert]
                try:
                    temp.append(str.strip(p.find("div", class_="rank").text))  #expert rank, number  .text
                    temp.append(str.strip(p.find("span", class_="player-name").text).replace(".", ""))  #cbs shortName  .text
                    temp.append(str.strip(p.find("a")["href"].split("/")[4])) # cbs playerId is in the url
                    temp.append(str.strip(p.find("span", class_="team position").text.split()[0])) # contains the player nfl team 
                    temp.append(pos) # contains the player nfl team
                    temp.append(np.nan)
                    temp.append(np.nan)
                    player_ranking_data.append(temp)
                except:
                    continue
        
    # creating temp dataframe that includes all 3 expert rankings for a grouping to add to the master df 
    temp_df = pd.DataFrame(player_ranking_data, columns=ranking_columns)        
    df_cbs_ranking = pd.concat([df_cbs_ranking, temp_df], axis = 0, ignore_index=True)
    
df_cbs_ranking.shape

In [ ]:
df_cbs_ranking.to_csv("../data/ranking/weekly/cbs_rank_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

# ----------- Fantasy Pros - HPPR -----------

They use CBS and ESPN for season stats

### Fantasy Pros Rankings

### Fantasy Pros ECRs

In [6]:
service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)
waitTime = 10

df_ecr_ranks = pd.DataFrame(columns=ranking_columns)

urls = [
    r"https://www.fantasypros.com/nfl/rankings/half-point-ppr-{}",
    r"https://www.fantasypros.com/nfl/rankings/{}"
] 
# try to close driver if there are any errors
try:
    for i in range(len(urls)):
        if i == 0:
            pos = ['SUPERFLEX', 'FLEX', 'TE', 'WR', 'RB']
            for j in pos:
                player_ranks = []
                driver.get(urls[i].format(j.lower()))

                # Accepting cookies if there is a popup
                try:
                    WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[contains(text(), 'Accept Cookies')]"))
                    cookies = driver.find_element("xpath", '//*[@id="onetrust-accept-btn-handler"]')
                    cookies.click()

                except: pass

                #driver.execute_script('videos = document.querySelectorAll("video"); for(video of videos) {video.pause()}')

                # select drop down that defaults to Overview and selecting Ranks
                WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[span[contains(text(), 'Overview')]]"))     
                drop = driver.find_element("xpath", "//button[span[contains(text(), 'Overview')]]")
                drop.click()

                WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[div[contains(text(), 'Ranks')]]"))     
                drop = driver.find_element("xpath", "//button[div[contains(text(), 'Ranks')]]")
                drop.click()

                # grab all html
                html = driver.page_source
                soup = bs(html, 'lxml')  #parse the html

                table = soup.find("table", id='ranking-table').find("tbody")
                ranks = table.find_all("tr")

                for tr in ranks:
                    tds = tr.find_all("td")
                    #for td in tds:
                        #print(td.text)

                    # some of the ecr defensive groups have teams in the rankings this will skip them
                    name = tds[2].text.split("(")[0].strip().replace(".", "")
                    if name in list(team_map.keys()):
                        continue

                    if j in ['SUPERFLEX', 'FLEX']:
                        rank = tds[0].text
                        team = tds[2].text.split("(")[1].strip().replace(")", "")
                        player = tds[2].find("div", class_='player-cell player-cell__td')['data-player']
                        high = tds[4].text
                        low = tds[5].text
                    else:
                        rank = tds[0].text
                        team = tds[2].text.split("(")[1].strip().replace(")", "")
                        player = tds[2].find("div", class_='player-cell player-cell__td')['data-player']
                        high = tds[4].text
                        low = tds[5].text


                    player_ranks.append(['fantasyPros', today, season, week, j, 'ecr', rank, name, player, team, np.nan,  high, low])

                temp = pd.DataFrame(player_ranks,  columns=ranking_columns)
                df_ecr_ranks = pd.concat([df_ecr_ranks, temp])

        if i == 1:
            pos = ['QB', 'K', 'DST', 'IDP', 'DL' ,'LB', 'DB']
            for j in pos:
                player_ranks = []
                driver.get(urls[i].format(j.lower()))

                # Accepting cookies if there is a popup
                try:
                    WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[contains(text(), 'Accept Cookies')]"))
                    cookies = driver.find_element("xpath", "//button[contains(text(), 'Accept Cookies')]")
                    cookies.click()

                except: pass

                # select drop down that defaults to Overview and selecting Ranks
                                                            # actual buttom xpath //*[@id="onetrust-accept-btn-handler"]
                WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[span[contains(text(), 'Overview')]]"))     
                drop = driver.find_element("xpath", "//button[span[contains(text(), 'Overview')]]")
                drop.click()

                WebDriverWait(driver, timeout=waitTime).until(lambda d: d.find_element("xpath", "//button[div[contains(text(), 'Ranks')]]"))     
                drop = driver.find_element("xpath", "//button[div[contains(text(), 'Ranks')]]")
                drop.click()

                # grab all html
                html = driver.page_source
                soup = bs(html, 'lxml')  #parse the html

                table = soup.find("table", id='ranking-table').find("tbody")
                ranks = table.find_all("tr")

                for tr in ranks:
                    tds = tr.find_all("td")

                    # some of the ecr defensive groups have teams in the rankings this will skip them
                    name = tds[2].text.split("(")[0].strip().replace(".", "")
                    if name in list(team_map.keys()):
                        continue

                    if j == 'IDP':
                        rank = tds[0].text
                        team = tds[2].text.split("(")[1].strip().replace(")", "")
                        player = tds[2].find("div", class_='player-cell player-cell__td')['data-player']
                        high = tds[4].text
                        low = tds[5].text

                    else:
                        rank = tds[0].text
                        team = tds[2].text.split("(")[1].strip().replace(")", "")
                        player = tds[2].find("div", class_='player-cell player-cell__td')['data-player']
                        high = tds[3].text
                        low = tds[4].text

                    player_ranks.append(['fantasyPros', today, season, week, j, 'ecr', rank, name, player, team, np.nan,  high, low])

                temp = pd.DataFrame(player_ranks, columns=ranking_columns)
                df_ecr_ranks = pd.concat([df_ecr_ranks, temp])
except Exception as ex:
    print(ex)
    driver.close()
    
driver.close()
df_ecr_ranks.shape

(1371, 13)

In [7]:
df_ecr_ranks.to_csv("../data/ranking/weekly/fpEcr_rank_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

### Fantasy Pros 5 Expert Rankings

In [8]:
url_fp_rankings = r"https://www.fantasypros.com/nfl/fantasy-football-rankings/{pos}.php"
                  
fp_url_positions = {"QB":"weekly-qb", 
                    "RB":"weekly-half-point-ppr-rb",
                    "WR":"weekly-half-point-ppr-wr", 
                    "TE":"weekly-half-point-ppr-te", 
                    "K":"weekly-k", 
                    "DST":"weekly-dst",
                    "FLEX":"weekly-half-point-ppr-flex", 
                    "SUPERFLEX":"weekly-half-point-ppr-superflex"}

all_ranks = []

for k,v in fp_url_positions.items(): 
    
    time.sleep(3)
    url_fp_formatted = url_fp_rankings.format(pos=v)
    r = requests.get(url_fp_formatted)
    soup = bs(r.text)
    
    # getting expert name and rank date
    experts = []
    for a in soup.find_all("th", class_="expert__th"):
        temp = []
        temp.append(a['data-sort-label'])   # expert name
        temp.append(str.strip(a.find("div", class_="expert__publish-date").text))  #ranking publish date
        experts.append(temp)

    # getting player info and ranks
    if k == "DST":
        for p in soup.find_all("tr", class_="player-row mpb-player__tr"):
            playerId = p["data-pid"]  # fp playerid
            shortName = p.find("span", class_="mobile-only").text.split()[0].replace(".", "") # fp short name
            #fullName = p.find("span", class_="everything-but-mobile js-sort-field").text # fp full name
            TEAM = p.find("span", class_="player__team").text  # player nfl team
            POS = p.find("span", class_="player__position").text  # player position

            html_ranks = p.find_all("td", attrs={'class': None})
            for r in range(len(html_ranks)):
                temp_fp_ranking = [shortName, playerId, TEAM, POS] 

                temp_fp_ranking.insert(0, html_ranks[r].text) # ranking
                temp_fp_ranking.insert(0, experts[r][0]) # expert
                temp_fp_ranking.insert(0, k) # group ranking set
                temp_fp_ranking.insert(0, week) #season week
                temp_fp_ranking.insert(0, season) #season year
                temp_fp_ranking.insert(0, experts[r][1]) # date
                temp_fp_ranking.insert(0, "fantasyPros") # outlet
                temp_fp_ranking.append(np.nan)
                temp_fp_ranking.append(np.nan)
                all_ranks.append(temp_fp_ranking)
    
    else:
        for p in soup.find_all("tr", class_="player-row mpb-player__tr"):
            playerId = p["data-pid"]  # fp playerid
            shortName = p.find("span", class_="mobile-only").text.replace(".", "") # fp short name
            #fullName = p.find("span", class_="everything-but-mobile js-sort-field").text # fp full name
            TEAM = p.find("span", class_="player__team").text  # player nfl team
            POS = p.find("span", class_="player__position").text  # player position

            html_ranks = p.find_all("td", attrs={'class': None})
            for r in range(len(html_ranks)):
                temp_fp_ranking = [shortName, playerId, TEAM, POS] 

                temp_fp_ranking.insert(0, html_ranks[r].text) # ranking
                temp_fp_ranking.insert(0, experts[r][0]) # expert
                temp_fp_ranking.insert(0, k) # group ranking set
                temp_fp_ranking.insert(0, week) #season week
                temp_fp_ranking.insert(0, season) #season year
                temp_fp_ranking.insert(0, experts[r][1]) # date
                temp_fp_ranking.insert(0, "fantasyPros") # outlet
                temp_fp_ranking.append(np.nan)
                temp_fp_ranking.append(np.nan)
                all_ranks.append(temp_fp_ranking)
                
df_fp_ranking = pd.DataFrame(all_ranks, columns=ranking_columns).replace("-", np.nan).replace("N/A", np.nan)
df_fp_ranking.shape

(6760, 13)

In [9]:
df_fp_ranking.to_csv("../data/ranking/weekly/fp_rank_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

# ----------- ESPN - HPPR -----------

### ESPN Projections

In [10]:
service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)

# open the initial projection page
url_espn_proj = "https://fantasy.espn.com/football/players/projections?leagueFormatId=3"
                
driver.get(url_espn_proj) 
# sleep to let the html load
time.sleep(10)


try:
    # changing to the desired projection view
    button = driver.find_element(By.XPATH, "//button[@class='Button Button--filter player--filters__projections-button']")
    button.click()
    time.sleep(5)
    
    # grabs the entire pages html
    html = driver.execute_script("return document.body.innerHTML")
    soup = bs(html)
    
    # grabbing the number of pages there are in the projections
    pagenation_list = soup.find("div", class_="Pagination__wrap overflow-x-auto")
    pages = pagenation_list.find_all("li")
    last_page = pages[-1].text
    
except Exception as ex:
    print(ex)
    driver.close()
    

espn_player_proj_player = []
page_count1 = 0
page_count2 = 0

for page in range(1, 18):  #int(last_page)+1):
    try:
        html = driver.execute_script("return document.body.innerHTML")
        soup = bs(html)

        # grabbing the projection tables
        tables = soup.find_all("table")
        
        # the player info table
        for tr in tables[0].find_all("tr"):
            for td in tr:
                if td.find("a", class_="AnchorLink link clr-link pointer"):
                    
                    #grabs the ESPN player id from the image url
                    playerId = td.find("img")['src'].split("/")[-1].split(".")[0]
                    #dst has player ID as the team abbreviation. This catches it
                    try:
                        int(playerId)
                    except:
                        playerId = ""
                        
                    name = td.find("a", class_="AnchorLink link clr-link pointer").text.replace(".", "")
                    position = td.find("span", class_="playerinfo__playerpos ttu").text.replace("/","").split(",")[0]
                    team = td.find("span", class_="playerinfo__playerteam").text.upper()

                    espn_player_proj_player.append(["espn", today, season, week, playerId, name, np.nan, position, team, np.nan])


        # the stat projection table
        for tr in tables[1].find_all("tr",class_="Table__TR Table__TR--lg Table__odd"):
            comp_att = tr.find("div", {"title":"Each Pass Completed & Each Pass Attempted"}).text.split("/")
            pass_comps = comp_att[0]
            pass_atts = comp_att[1]
            pass_yds = tr.find("div", {"title":"Passing Yards"}).text
            pass_tds = tr.find("div", {"title":"TD Pass"}).text
            ints = tr.find("div", {"title":"Interceptions Thrown"}).text
            rush_atts = tr.find("div", {"title":"Rushing Attempts"}).text
            rush_yds = tr.find("div", {"title":"Rushing Yards"}).text
            rush_tds = tr.find("div", {"title":"TD Rush"}).text
            rec = tr.find("div", {"title":"Each reception"}).text
            rec_yds = tr.find("div", {"title":"Receiving Yards"}).text
            rec_tds = tr.find("div", {"title":"TD Reception"}).text
            rec_trgts = tr.find("div", {"title":"Receiving Target"}).text
            
            espn_player_proj_player[page_count1].extend([pass_atts, pass_comps,pass_yds, 0, pass_tds,
                                                         ints, 0, rush_atts,rush_yds,0, rush_tds,rec_trgts,rec,rec_yds,0,0,rec_tds,
                                                        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
            page_count1 += 1
       
        # the fantasy points table
                                        
        for tr in tables[2].find_all("tr",class_="Table__TR Table__TR--lg Table__odd"):
            for div in tr.find_all("div"):
                # some of the free agents/retired players don't have div["title"] need to catch them with try
                try:
                    if 'point' in div['title']:
                        total_ff_pts = div.find("span").text
                        avg_ff_pts = 0
                except:
                    total_ff_pts = 0
                    avg_ff_pts = 0

            espn_player_proj_player[page_count2].extend([total_ff_pts, avg_ff_pts])
            page_count2 += 1
        
        #checks for last page
        
        if page < int(last_page):
            # jumping to the next page
            nextButton = driver.find_element(By.XPATH, "//button[@class='Button Button--default Button--icon-noLabel Pagination__Button Pagination__Button--next']")
            nextButton.click()
            time.sleep(10)
            
    except Exception as ex:
        print(ex)
        driver.close()

try:
    driver.close()
except:
    pass

# creating df from gathered data to merge into final df that matches the cbs structure
temp_proj = pd.DataFrame(espn_player_proj_player, columns = projection_columns)
                        

df_espn_proj = pd.DataFrame(columns = projection_columns)

#final espn projections data
df_espn_proj = pd.concat([df_espn_proj, temp_proj]).replace("--", 0)
df_espn_proj.shape

(850, 60)

In [11]:
df_espn_proj.to_csv("../data/projection/weekly/espn_proj_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

### ESPN Rankings  - need to update urls for each season

In [13]:
espn_ranking_urls = {
"QB":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankQBPPR/nfl-fantasy-football-rankings-2023-qb-quarterback",
"RB":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankRBPPR/nfl-fantasy-football-rankings-2023-rb-running-back",
"WR":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankWRPPR/nfl-fantasy-football-rankings-2023-wr-wide-receiver",
"TE":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankTEPPR/nfl-fantasy-football-rankings-2023-te-tight-end",
"K":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankKPPR/nfl-fantasy-football-rankings-2023-kicker",
"DST":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankDSTPPR/nfl-fantasy-football-rankings-2023-dst-defense",
"IDP":"https://www.espn.com/fantasy/football/story/_/page/23ffweekrankIDP/nfl-fantasy-football-rankings-2023-idp-defense-dl-lb-db",
}

# final dataframe structure to hosue all the rankings
df_espn_ranking = pd.DataFrame(columns=ranking_columns)

service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)
#try to closed driver if there are any errors
try:
    # looping through the urls to aggregate the rankings
    for group, url in espn_ranking_urls.items():
        # opening the webpage and allowing the scripts to load for the HTML to be accessed

        url_espn_formatted = url.format(season=season)
        driver.get(url_espn_formatted)
        time.sleep(10)

        # grabs the entire pages html
        html = driver.execute_script("return document.body.innerHTML")
        soup = bs(html)

        # this will hold a list of list. One list will be a players rank for a single expert
        player_ranks = []

        # IDP page has 3 separate tables for positions instead of a single position on the page and a single table handled in the else below
        if group == "IDP":
            ranking_tables = soup.find_all("table", class_="inline-table rankings-table fullWidth sortable")
            count = 0 # hard coded the positions based on the which table the site holds them in

            # 3 tables for the 3 IDPs  DL, LB, DB
            for ranking_table in ranking_tables:

                # retrieves the expert names and the order they are listed
                expert_names_html = ranking_table.find("thead").find_all("th")
                expert_names = []
                for tr in range(2, len(expert_names_html)-1):
                    expert_names.append(expert_names_html[tr].text)

                player_ranks_html = ranking_table.find("tbody").find_all("tr", class_="")
                for tr in player_ranks_html:

                    tds = tr.find_all("td")

                    playerId = tds[0].find("a")["data-player-id"]
                    name = tds[0].find("a").text.replace(".", "")

                    if count == 0:
                        POS = "DL"
                    elif count == 1:
                        POS = "LB"
                    elif count == 2:
                        POS = "DB"

                    # try block to handle injury designations that the site puts in the same text as the team name
                    try:
                        #if there is a injury designation, it retrieves it and then removes it from the team name
                        injury = tds[0].find_all("div", class_="rank")[0].find("span").text
                        if len(injury) > 1:  # Accounts for suspended tag "SSPD"
                            team = tds[0].find_all("div", class_="rank")[0].text.split(",")[1].strip().upper()[:-4]
                        else:
                            team = tds[0].find_all("div", class_="rank")[0].text.split(",")[1].strip().upper()[:-1]

                    except:
                        team = tds[0].find("div", class_="rank").text.split(",")[1].strip().upper()

                    for i in range(len(expert_names)):

                        # expert name from the list generated from thead
                        expert = expert_names[i]
                        # position of the expert ranking column in tbody
                        idx = i + 2

                        # retrieves the expert rank from tbody rows
                        exRank = pd.to_numeric(tds[idx].text, errors='coerce')

                        player_ranks.append(["espn", today, season, week, POS, expert, exRank, name, playerId, team,  POS, np.nan, np.nan])

                count += 1


        # for position specific rankings
        else:

            ranking_table = soup.find("table", class_="inline-table rankings-table fullWidth sortable")
            #driver.close()

            # retrieves the expert names and the order they are listed
            expert_names_html = ranking_table.find("thead").find_all("th")
            expert_names = []
            for tr in range(2, len(expert_names_html)-1):
                expert_names.append(expert_names_html[tr].text)

            player_ranks_html = ranking_table.find("tbody").find_all("tr", class_="")
            for tr in player_ranks_html:

                tds = tr.find_all("td")

                playerId = tds[0].find("a")["data-player-id"]
                if group == "DST":
                    name = tds[0].find("a").text.split()[0].replace(".", "")
                else:
                    name = tds[0].find("a").text.replace(".", "")

                POS = group

                #team = tds[0].find("div", class_="rank").text.split(",")[1].strip().upper()
                # try block to handle injury designations that the site puts in the same text as the team name
                try:
                    #if there is a injury designation, it retrieves it and then removes it from the team name
                    injury = tds[0].find_all("div", class_="rank")[0].find("span").text
                    if len(injury) > 1:  # Accounts for suspended tag "SSPD"
                        team = tds[0].find_all("div", class_="rank")[0].text.split(",")[1].strip().upper()[:-4]
                    else:
                        team = tds[0].find_all("div", class_="rank")[0].text.split(",")[1].strip().upper()[:-1]

                except:
                    team = tds[0].find("div", class_="rank").text.split(",")[1].strip().upper()

                for i in range(len(expert_names)-1):

                    # expert name from the list generated from thead
                    expert = expert_names[i]
                    # position of the expert ranking column in tbody
                    idx = i + 2

                    # retrieves the expert rank from tbody rows
                    exRank = pd.to_numeric(tds[idx].text, errors='coerce')

                    player_ranks.append(["espn", today, season, week, group, expert, exRank, name, playerId, team,  POS, np.nan, np.nan])


        temp_df = pd.DataFrame(player_ranks, columns=ranking_columns)
        df_espn_ranking = pd.concat([df_espn_ranking, temp_df], axis = 0, ignore_index=True)
    
except Exception as ex:
    print(ex)
    driver.close()    
    
driver.close() 
df_espn_ranking.shape

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_13028\1680112104.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_espn_ranking = pd.concat([df_espn_ranking, temp_df], axis = 0, ignore_index=True)


(1850, 13)

In [14]:
df_espn_ranking.to_csv("../data/ranking/weekly/espn_rank_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

# ------------ NFL --------------

### NFL projections

In [15]:
# position=  0:QB,RB,WR,TE  7:Kicker, 8:D
nfl_proj_url = [
"https://fantasy.nfl.com/research/projections?offset={offset}&position=0&statCategory=projectedStats&statSeason={season}&statType=weekProjectedStats&statWeek={week}",
"https://fantasy.nfl.com/research/projections?offset={offset}&position=7&statCategory=projectedStats&statSeason={season}&statType=weekProjectedStats&statWeek={week}",
"https://fantasy.nfl.com/research/projections?offset={offset}&position=8&statCategory=projectedStats&statSeason={season}&statType=weekProjectedStats&statWeek={week}",
]


df_nfl_proj = pd.DataFrame(columns=projection_columns)
player_data = []
# count will be updated to the player count after the first page load 
# this is being used to avoid loading more pages than needed
count = 3000

#looping through the 3 URLs, the site has QB,RB,WR,TE combined in a single list and then K and D on their own pages
for i in range(3):
    if i == 0:  # this will handle the offensive players
        while count > 25:
            
            # this grabs the first page, else will handle all others
            if count == 3000:
                time.sleep(1)
                r = requests.get(nfl_proj_url[0].format(offset=1, season=season, week=week))
                soup = bs(r.text)

                # grabs the number of players with projections on the site. pagenated at 25 a page
                player_count = int(soup.find("span", class_="paginationTitle").text.split("of")[-1].strip())
                count = player_count

                table = soup.find_all("table", class_="tableType-player hasGroups")

                body_trs = table[0].find("tbody").find_all("tr")

                for tr in body_trs:
                    data = tr.find_all("td")

                    firstColA = data[0].find('a')
                    playerId = firstColA['href'].split("=")[2]
                    fullName = firstColA.text.strip().replace(".", "")

                    posAndTeam = data[0].find('em').text.split("-")
                    pos = posAndTeam[0].strip()
                    try:
                        team = posAndTeam[1].strip()
                    except:
                        team = "FA"

                    
                    PassingYards = data[2].text
                    TouchdownsPasses = data[3].text
                    InterceptionsThrown = data[4].text
                    RushingYards = data[5].text
                    RushingTouchdowns = data[6].text
                    Receptions = data[7].text
                    ReceivingYards = data[8].text
                    ReceivingTouchdowns = data[9].text
                    retTd = data[10].text
                    fumTd = data[11].text
                    twoPt= data[12].text
                    FumblesLost = data[13].text
                    FantasyPoints = data[14].text

                    temp = ["nfl", today, season, week, playerId,fullName,np.nan,pos,team,0,0,0,PassingYards,0,TouchdownsPasses, InterceptionsThrown,
                            0,0,RushingYards,0,RushingTouchdowns,0,Receptions,ReceivingYards,0,0,ReceivingTouchdowns,
                            FumblesLost,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,twoPt,FantasyPoints,0]
                    player_data.append(temp)

            else:
                for j in range(26, player_count, 25):
                    time.sleep(1)
                    r = requests.get(nfl_proj_url[0].format(offset=j, season=season, week=week))
                    soup = bs(r.text)
                    table = soup.find_all("table", class_="tableType-player hasGroups")
                    body_trs = table[0].find("tbody").find_all("tr")

                    for tr in body_trs:
                        data = tr.find_all("td")

                        firstColA = data[0].find('a')
                        playerId = firstColA['href'].split("=")[2]
                        fullName = firstColA.text.strip().replace(".", "")

                        posAndTeam = data[0].find('em').text.split("-")
                        pos = posAndTeam[0].strip()
                        try:
                            team = posAndTeam[1].strip()
                        except:
                            team = "FA"

                        PassingYards = data[2].text
                        TouchdownsPasses = data[3].text
                        InterceptionsThrown = data[4].text
                        RushingYards = data[5].text
                        RushingTouchdowns = data[6].text
                        Receptions = data[7].text
                        ReceivingYards = data[8].text
                        ReceivingTouchdowns = data[9].text
                        retTd = data[10].text
                        fumTd = data[11].text
                        twoPt= data[12].text
                        FumblesLost = data[13].text
                        FantasyPoints = data[14].text

                        temp = ["nfl", today, season, week, playerId,fullName,np.nan,pos,team,0,0,0,PassingYards,0,TouchdownsPasses, InterceptionsThrown,
                                0,0,RushingYards,0,RushingTouchdowns,0,Receptions,ReceivingYards,0,0,ReceivingTouchdowns,
                                FumblesLost,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,twoPt,FantasyPoints,0]
                        player_data.append(temp)

                    count -= 25
                    
    else: # this will handle K and D
        for j in range(2):  
            
            time.sleep(1)
            r = requests.get(nfl_proj_url[i].format(offset=j*25+1, season=season, week=week))  # k and d only have 2 pages, j *25 + 1 handles the url offset that pagenates
            soup = bs(r.text)

            table = soup.find_all("table", class_="tableType-player hasGroups")

            body_trs = table[0].find("tbody").find_all("tr")

            for tr in body_trs:
                data = tr.find_all("td")
                temp = []
                
                firstColA = data[0].find('a')
                playerId = firstColA['href'].split("=")[2]
                fullName = firstColA.text.strip().replace(".", "")

                posAndTeam = data[0].find('em').text.split("-")
                
                
                if i == 1:  # K url
                    
                    pos = posAndTeam[0].strip()
                    try:
                        team = posAndTeam[1].strip()
                    except:
                        team = "FA"
                        
                    xpMade = data[2].text
                    made0_19 = float(data[3].text.replace("-", "0"))
                    made20_29 = float(data[4].text.replace("-", "0"))
                    made30_39 = float(data[5].text.replace("-", "0"))
                    made40_49 = float(data[6].text.replace("-", "0"))
                    made50 = float(data[7].text.replace("-", "0"))
                    fgMade = made0_19 + made20_29 + made30_39 + made40_49 + made50
                    FantasyPoints = data[8].text

                    temp = ["nfl", today, season, week, playerId,fullName,np.nan,pos,team,0,0,0,0,0,0, 0,
                                    0,0,0,0,0,0,0,0,0,0,0,0,
                                    fgMade,0,0,made0_19,0,made20_29,0,made30_39,0,made40_49,0,made50,0,xpMade,0,0,
                                     0,0,0,0,0,0,0,0,0,0,0,0,0,0,FantasyPoints,0]
                    player_data.append(temp)    
                        
                else: # D url
                    pos = 'DST'
                    team = fullName
                    sacks = data[2].text
                    interceptions = data[3].text
                    fum = data[4].text
                    safety = data[5].text
                    defTd = data[6].text
                    twoPt = data[7].text
                    retTd = data[8].text
                    ptsAllowed= data[9].text
                    fantasyPts= data[10].text
                    
                        
                    
                    temp = ["nfl", today, season, week, playerId,np.nan,np.nan,pos,team,0,0,0,0,0,0,0,
                            0,0,0,0,0,0,0,0,0,0,0,
                            FumblesLost,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,interceptions,safety,sacks,0,fum,0,
                            defTd, retTd,ptsAllowed,0,0,0,0,0,twoPt,fantasyPts,0]
                    
                    player_data.append(temp)
        
df_nfl_proj = pd.DataFrame(player_data, columns=projection_columns).replace("-",0)
df_nfl_proj.shape

In [16]:
df_nfl_proj.to_csv("../data/projection/weekly/nfl_proj_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

### NFL rankings

In [18]:
nfl_rank_url = {
    "QB":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=QB&sort=1&statType=weekStats&week={week}",     
    "RB":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=RB&sort=1&statType=weekStats&week={week}",
    "WR":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=WR&sort=1&statType=weekStats&week={week}",
    "TE":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=TE&sort=1&statType=weekStats&week={week}",
    "K":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=K&sort=1&statType=weekStats&week={week}",
    "DST":"https://fantasy.nfl.com/research/rankings?leagueId=0&position=DEF&sort=1&statType=weekStats&week={week}"
}

df_nfl_ranking = pd.DataFrame(columns=ranking_columns)

for k,v in nfl_rank_url.items():
    
    time.sleep(1)
    r = requests.get(nfl_rank_url[k].format(week=week))
    soup = bs(r.text)

    # grabs the number of players with projections on the site. pagenated at 25 a page
    rank_table = soup.find("table", class_="tableType-player noGroups").find("tbody")

    player_ranks = []
    for tr in rank_table.find_all("tr"):
        player_data = []
        td = tr.find_all("td")
        
        pos_rank = int(td[0].text)
        playerId = int(td[1].find("a")['href'].split("=")[-1])
        full_name = td[1].find("a").text.replace(".", "")
        
        pos = td[1].find("em").text.split("-")[0].strip()
        
        if k == "DST":
            team = ""
            pos = "DST"
        
        else:
            # no team name for FAs
            try:
                team = td[1].find("em").text.split("-")[1].strip()
            except:
                team = "FA"
            
        ovr_rank = int(td[-1].text)

        player_data = ["nfl", today, season, week, k, "nfl", pos_rank, full_name, playerId, team, pos, np.nan, np.nan]
        player_ranks.append(player_data)

    temp_df = pd.DataFrame(player_ranks, columns=ranking_columns)
    df_nfl_ranking = pd.concat([df_nfl_ranking, temp_df], axis = 0, ignore_index=True)
    
df_nfl_ranking.shape

(214, 13)

In [19]:
df_nfl_ranking.to_csv("../data/ranking/weekly/nfl_rank_{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), index=False)

### Weekly Bettings Lines from Betting Pros

In [36]:
service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)


driver.get(urls[0].format(season=season, week=week)) 
# sleep to let the html load
time.sleep(10)

html = driver.execute_script("return document.body.innerHTML")
soup = bs(html)

data = soup.find_all("div", class_="flex odds-offer")

matchups = []
driver.close()

In [52]:
data_team = data[0].find_all("a", class_="team-overview__team-name")
data_spread = data[0].find_all("div", class_="flex odds-offer__item")

# team 1 data - away
away_team = data_team[0]['href'].split("/")[3].replace("-", " ").title().strip()
away_spread_line = data_spread[2].find_all("span", class_="odds-cell__line")#[1].text.replace("+", "")

data_spread[0].find_all("span", class_="odds-cell__cost")

[<span class="typography odds-cell__cost" data-v-e0bd5419="" data-v-f8c6c06d="" style="--64dcb17e: #16191D; --0b8a2e86: 1.1rem; --584c03bd: 600; --faa089fc: left; --97ef6e2e: none; --da5e331a: normal;">(-110)</span>,
 <span class="typography odds-cell__cost" data-v-e0bd5419="" data-v-f8c6c06d="" style="--64dcb17e: #16191D; --0b8a2e86: 1.1rem; --584c03bd: 600; --faa089fc: left; --97ef6e2e: none; --da5e331a: normal;">(-110)</span>]

In [55]:
service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)

urls = [
    r'https://www.bettingpros.com/nfl/odds/spread/?season={season}&week={week}',
    r"https://www.bettingpros.com/nfl/odds/moneyline/?season={season}&week={week}",
    r"https://www.bettingpros.com/nfl/odds/total/?season={season}&week={week}"
]
  
# empty, to be filled with data from websites
df_lines = pd.DataFrame(columns=["date", "season", "week", "overUnder", "overUnderCost" 
                                                "awayTeam", "awaySpread", "awayCost", "awayMoneyline",  
                                                "homeTeam", "homeSpread", "homeCost", "homeMoneyLine"]) 
# try to closed driver on error
try:
    for i in range(len(urls)):
        driver.get(urls[i].format(season=season, week=week)) 
        # sleep to let the html load
        time.sleep(10)

        html = driver.execute_script("return document.body.innerHTML")
        soup = bs(html)

        data = soup.find_all("div", class_="flex odds-offer")

        matchups = []
        # spreads
        if i == 0:

            for div in data:

                data_team = div.find_all("a", class_="team-overview__team-name")
                data_spread = div.find_all("div", class_="flex odds-offer__item")

                # team 1 data - away
                away_team = data_team[0]['href'].split("/")[3].replace("-", " ").title()
                away_spread_line = data_spread[0].find_all("span", class_="odds-cell__line")[0].text.replace("+", "")

                if (away_spread_line == "NL") or (away_spread_line == "--"):
                    away_spread_line = np.nan

                away_spread_cost = data_spread[0].find_all("span", class_="odds-cell__cost")[0].text.strip().replace("+", "").replace("(", "").replace(")", "")
                if (away_spread_cost == "NL") or (away_spread_cost == "--"):
                    away_spread_cost = np.nan

                #team 2 data - home
                home_team = data_team[1]['href'].split("/")[3].replace("-", " ").title()
                home_spread_line = data_spread[0].find_all("span", class_="odds-cell__line")[1].text.replace("+", "")

                if (home_spread_line == "NL") or (home_spread_line == "--"):
                    home_spread_line = np.nan

                home_spread_cost = data_spread[0].find_all("span", class_="odds-cell__cost")[1].text.strip().replace("+", "").replace("(", "").replace(")", "")
                if (home_spread_cost == "NL") or (home_spread_cost == "--"):
                    home_spread_cost = np.nan

                matchup = [today, season, week,
                           np.nan, np.nan,  # placeholder for overUnders
                           away_team, float(away_spread_line), away_spread_cost, np.nan,
                           home_team, float(home_spread_line), home_spread_cost, np.nan
                          ]

                matchups.append(matchup)

            df_lines = pd.DataFrame(matchups, columns=["date", "season", "week", "overUnder", "overUnderCost",
                                                    "awayTeam", "awaySpread", "awayCost", "awayMoneyline",  
                                                    "homeTeam", "homeSpread", "homeCost", "homeMoneyLine"])

        #moneylines
        elif i == 1:
            for div in data:

                data_team = div.find_all("a", class_="team-overview__team-name")
                data_moneylines = div.find_all("div", class_="flex odds-offer__item")

                # team 1 data - away
                away_team = data_team[0]['href'].split("/")[3].replace("-", " ").title()
                away_moneyline = data_moneylines[1].find_all("span", class_="odds-cell__line")[0].text.strip().replace("+", "")
                if (away_moneyline == "NL") or (away_moneyline == "--"):
                    away_moneyline = np.nan
                elif away_moneyline == "EVEN":
                    away_moneyline = 0
                else:
                    away_moneyline = float(away_moneyline)

                #team 2 data - home
                home_moneyline = data_moneylines[1].find_all("span", class_="odds-cell__line")[1].text.strip().replace("+", "")
                if (home_moneyline == "NL") or (home_moneyline == "--"):
                    home_moneyline = np.nan
                elif home_moneyline == "EVEN":
                    home_moneyline = 0
                else:
                    home_moneyline = float(home_moneyline)

                df_lines.loc[df_lines[df_lines['awayTeam'] == away_team].index[0],'awayMoneyline'] = away_moneyline
                df_lines.loc[df_lines[df_lines['awayTeam'] == away_team].index[0],'homeMoneyLine'] = home_moneyline

        # totals
        elif i == 2:

            for div in data:

                data_team = div.find_all("a", class_="team-overview__team-name")
                data_spread = div.find_all("div", class_="flex odds-offer__item")

                # team 1 data - away
                away_team = data_team[0]['href'].split("/")[3].replace("-", " ").title()

                overUnder_line = data_spread[0].find_all("span", class_="odds-cell__line")[0].text.strip()
                if (overUnder_line == "NL") or (overUnder_line == "--"):
                    overUnder_line = np.nan
                else:
                    overUnder_line = float(overUnder_line.replace("+", "").split(" ")[1])



                df_lines.loc[df_lines[df_lines['awayTeam'] == away_team].index[0],'overUnder'] = overUnder_line
                df_lines.loc[df_lines[df_lines['awayTeam'] == away_team].index[0],'overUnderCost'] = -110

    df_lines = df_lines.replace(np.nan, None).replace("EVEN", 100)
    df_lines.to_csv("../data/betting/weekly/lines{season}-{week}_{date}.csv".format(season=season, week=strWeek, date=today), 
                    index=False, )

except Exception as ex:
    print(ex)
    driver.close()

driver.close()  

df_lines.shape

(16, 13)

In [ ]:
df_lines

# END OF WEEK SCORES AND STATS

In [23]:
url_scores = r'https://www.cbssports.com/nfl/scoreboard/all/{season}/regular/{week}/'
prev_week = week - 1
# retreiving HTML and converting it to soup
r = requests.get(url_scores.format(season=season, week=str(prev_week)))
soup = bs(r.text)

# accessing tables with the data
tables = soup.find_all("div", class_="live-update")


records = pd.DataFrame(columns = ["season", "week", "team", "wins", "losses", "ties"])
games = pd.DataFrame(columns=["season", "week", "homeTeam", "homeQ1Pts", "homeQ2Pts", "homeQ3Pts", "homeQ4Pts", "homeTotalPts",
        "awayTeam", "awayQ1Pts", "awayQ2Pts", "awayQ3Pts", "awayQ4Pts", "awayTotalPts",
        "homeWinner", "awayWinner", "tie", "winningTeam", "gameId"])

for t in tables:
    game = t.find("tbody").find_all("tr")

    # away team 
    away = game[0]
    away_team = away.find("a")['href'].split("/")[3]
    away_record = away.find("span", class_="record").text.split("-")
    away_win = away_record[0]
    away_loss = away_record[1]
    if len(away_record) == 3:
        away_tie = away_record[2]
    else: 
        away_tie = 0
    away_scores = away.find_all("td")
    away_q1Pts = int(away_scores[1].text)
    away_q2Pts = int(away_scores[2].text)
    away_q3Pts = int(away_scores[3].text)
    away_q4Pts = int(away_scores[4].text)
    away_totalPts = int(away_scores[5].text)

    # home team 
    home = game[1]
    home_team = home.find("a")['href'].split("/")[3]
    home_record = home.find("span", class_="record").text.split("-")
    home_win = home_record[0]
    home_loss = home_record[1]
    if len(home_record) == 3:
        home_tie = home_record[2]
    else: 
        home_tie = 0
    home_scores = home.find_all("td")
    home_q1Pts = int(home_scores[1].text)
    home_q2Pts = int(home_scores[2].text)
    home_q3Pts = int(home_scores[3].text)
    home_q4Pts = int(home_scores[4].text)
    home_totalPts = int(home_scores[5].text)

    # designating winner or tie
    home_winner = 0
    away_winner = 0
    tie = 0
    winning_team = np.nan

    if home_totalPts > away_totalPts:
        home_winner = 1
        winning_team = home_team
    elif home_totalPts < away_totalPts:
        away_winner = 1
        winning_team = away_team
    else:
        tie = 1
    
    # gathering data into list for df creation and creating temp dfs to concat with main df
    home_records = [season, prev_week, home_team, home_win, home_loss, home_tie]
    away_records = [season, prev_week, away_team, away_win, away_loss, away_tie]
    game_data = [season, week-1, home_team, home_q1Pts, home_q2Pts, home_q3Pts, home_q4Pts, home_totalPts,
                 away_team, away_q1Pts, away_q2Pts, away_q3Pts, away_q4Pts, away_totalPts,
                home_winner, away_winner, tie, winning_team]

     
    temp_games = pd.DataFrame([game_data], columns=["season", "week", "homeTeam", "homeQ1Pts", "homeQ2Pts", "homeQ3Pts", "homeQ4Pts", "homeTotalPts",
        "awayTeam", "awayQ1Pts", "awayQ2Pts", "awayQ3Pts", "awayQ4Pts", "awayTotalPts",
        "homeWinner", "awayWinner", "tie", "winningTeam"])
    
    cols = ['season', 'week', 'homeTeam', 'awayTeam']
    temp_games['gameId'] = temp_games[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    temp_games = temp_games.replace("WAS", "WSH").replace("JAC", "JAX")
    
    games = pd.concat([games, temp_games])
    
    temp_records_h = pd.DataFrame([home_records], columns = ["season", "week", "team", "wins", "losses", "ties"])
    temp_records_a = pd.DataFrame([away_records], columns = ["season", "week", "team", "wins", "losses", "ties"])
    temp_records_h = pd.concat([temp_records_h, temp_records_a])
    temp_records_h = temp_records_h.replace("WAS", "WSH").replace("JAC", "JAX")
    
    records = pd.concat([records, temp_records_h])

# INSERT DATA INTO DATABASE

#### CONNECT TO DB

In [75]:
# import credentials
dbUser = creds['mysqlSurface']['users'][1]
dbPw = creds['mysqlSurface']['creds']['jb']
dbHost = creds['mysqlSurface']['dbNFL']['host']
dbName = creds['mysqlSurface']['dbNFL']['database']
dbConnectionString = creds['pymysql'][league]

In [76]:
# creating lookup dictionaries that will be used across multiple database inserts
try:
    engine = create_engine(dbConnectionString)
    conn = engine.connect()
    
    # getting outlet db ids to convert the scraped names/ids
    outletLookup = pd.read_sql("SELECT outletId, outletName  FROM outlet;", conn)
    outletLookup = pd.Series(outletLookup.outletId.values, index=outletLookup.outletName).to_dict()
    
    # getting team db ids to convert datasource names to the ids
    teamLookup = pd.read_sql("SELECT teamId, name  FROM team;", conn)
    teamLookup = pd.Series(teamLookup.teamId.values, index=teamLookup.name).to_dict()
    
    # getting expert db ids to convert the scraped names/ids
    expertLookup = pd.read_sql("SELECT analystId, analystName  FROM analyst;", conn)
    expertLookup = pd.Series(expertLookup.analystId.values, index=expertLookup.analystName).to_dict()

    
    # create name table back up
    names = pd.read_sql(con=conn, sql="SELECT * FROM player")
    names.to_csv("../data/names_backup.csv", index=False)
    
    conn.close()
    
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

#### LOAD RANKINGS

In [77]:
try:
    #main df to hold all data
    df_load_rank = pd.DataFrame(columns = [
        'outlet', 'date', 'season', 'week', 'group', 'expert', 'rank', 'high', 'low','playerId'
    ])

    # df to hold players that are not in the database for a source yet
    df_missing_players_rank = pd.DataFrame(columns=['date', 'outlet', 'group', 'playerId', 'sourceId', 'name'])

    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:

        # combining all outlets rankings to a single dataframe and converting names to the database Ids
        directory = r'../data/ranking/weekly'
        #looping through every rank file to aggregate into single df
        for filename in os.listdir(directory):
            f = os.path.join(directory,filename)
            # checking if it is a file
            if os.path.isfile(f):
                temp = pd.read_csv(f, parse_dates=['date'], infer_datetime_format=True,
                                   ###################################################################
                                   names=['outlet','date', 'season', 'week', 'group', 'expert', 'rank', 
                                          'name', 'sourceId', 'team', 'pos', 'high', 'low', 'playerId'],
                          skiprows=1)
                            ##########################################################################

                # updtaing outlet specific playerIds to database IDs
                if 'cbs_' in f:
                    lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

                elif 'espn_' in f:
                    lookup = pd.read_sql("SELECT playerId, espnId FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()

                elif ('fp_' in f) or ('fpEcr_' in f):
                    lookup = pd.read_sql("SELECT playerId, fpId FROM player WHERE fpId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.fpId).to_dict()

                elif 'nfl_' in f:
                    lookup = pd.read_sql("SELECT playerId, nflId FROM player WHERE nflId IS NOT NULL;", conn)
                    lookup['nflId'] = pd.to_numeric(lookup['nflId']) 
                    lookup = pd.Series(lookup.playerId.values, index=lookup.nflId).to_dict()

                # using the lookup to make the change from outletId to dbId
                temp['playerId'] = temp['sourceId'].map(lookup)

                ####################################
                # creating a df to hold that date for players who are not in the player table for the source
                if temp[pd.isnull(temp['playerId'])].shape[0] > 0:
                    df_missing_players_rank = pd.concat([df_missing_players_rank, 
                                                    temp.loc[pd.isnull(temp['playerId']), ['date', 'outlet', 'group', 'playerId', 'sourceId', 'name']]])
                ####################################

                # updating outlet name to db outlet id 
                temp['outlet'] = temp['outlet'].replace(outletLookup)

                #updating expert name to db expert id
                temp['expert'] = temp['expert'].replace(expertLookup)

                temp = temp[['outlet', 'date', 'season', 'week', 'group', 'expert', 'rank', 'high', 'low', 'playerId']]
                # adding outlet dataframe to the upload dataframe
                df_load_rank = pd.concat([df_load_rank, temp])

                
    df_load_rank = df_load_rank.replace(np.nan, None)
    # removing unranked players and rankings that have been loaded already
    df_load_rank = df_load_rank.loc[pd.notnull(df_load_rank['rank'])]
    df_load_rank = df_load_rank.loc[df_load_rank['date'] >= pd.to_datetime(today)]



except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_13028\4255197158.py:20: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  temp = pd.read_csv(f, parse_dates=['date'], infer_datetime_format=True,
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_13028\4255197158.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_missing_players_rank = pd.concat([df_missing_players_rank,
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_13028\4255197158.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. 

In [78]:
if df_missing_players_rank.shape[0] > 0:
    df_missing_players_rank.to_csv('../data/missingPlayersRank.csv')
df_missing_players_rank.shape

(1316, 6)

In [38]:
try:
    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
        
        # insert using SQLalchemy
        data = df_load_rank.replace(np.nan, None).values.tolist()
        rankMeta = sal.MetaData(engine)
        rankTable = sal.Table('ranking', rankMeta, autoload=True)
        conn.execute(sal.insert(rankTable).values(data))
    
    ''' old mysql.connector connection
        query_insert = """
            INSERT INTO ranking 
                (outletId, date, season, week, rankGroup, analystId, ranking, high, low, playerId)
            VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

        cursor.executemany(query_insert, df_load_rank.values.tolist())
        conn.commit()
        conn.close()
        '''
    print("success")
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)
    

success


In [41]:
action = 'update'
source = 'cbs'

db_action = {
                'update':{
                        "espn": """UPDATE player SET espnId = %s,espnName = %s WHERE playerId = %s""",
                         "cbs": """UPDATE player SET cbsId = %s,cbsName = %s WHERE playerId = %s""",
                         "fp": """UPDATE player SET fpId = %s,fpName = %s WHERE playerId = %s""",
                         "nfl": """UPDATE player SET nflId = %s,nflName = %s WHERE playerId = %s"""
                },
             'add':{
                         "espn": """INSERT INTO player (posId,teamId,espnId,espnName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "cbs": """INSERT INTO player (posId,teamId,cbsId,cbsName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "fp": """INSERT INTO player (posId,teamId,fpId,fpName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "nfl": """INSERT INTO player (posId,teamId,nflId,nflName,name) VALUES (%s,%s,%s,%s,%s)"""
             },
            'delete':{
                
            }
}

db_action[action][source]

'UPDATE player SET cbsId = %s,cbsName = %s WHERE playerId = %s'

In [42]:
#####
## THIS CAN BE USED TO UPDATE THE DB FOR MISSING PLAYERS
#####
df_missing = pd.read_excel(r'../data/adds.xlsx', sheet_name=action)

try:
    #engine = create_engine(dbConnectionString)
    #conn = engine.connect()
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    cursor.executemany(db_action[action][source], df_missing.values.tolist())
        
    conn.commit()
    conn.close()
    print("success")
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

success


#### LOAD PROJECTIONS

In [46]:
try:  
    projection_columns_dbLoad = ["playerId", "date", "season", "week", "outlet",'GamesPlayed',
     'PassAttempts','PassCompletions','PassingYards', 'PassingYardsPerGame', 'TouchdownsPasses', 
     'InterceptionsThrown', 'PasserRating','RushingAttempts','RushingYards', 'AverageYardsPerRush', 'RushingTouchdowns',
     'Targets', 'Receptions', 'ReceivingYards', 'YardsPerGame', 'AverageYardsPerReception','ReceivingTouchdowns',
     'FumblesLost','FieldGoalsMade','FieldGoalAttempts','LongestFieldGoal','FieldGoals119Yards','FieldGoals119YardAttempts',
     'FieldGoals2029Yards','FieldGoals2029YardAttempts','FieldGoals3039Yards','FieldGoals3039YardAttempts',
     'FieldGoals4049Yards','FieldGoals4049YardAttempts','FieldGoals50Yards','FieldGoals50YardsAttempts',
     'ExtraPointsMade','ExtraPointsAttempted','Interceptions','Safeties','Sacks','Tackles','DefensiveFumblesRecovered',
     'ForcedFumbles','DefensiveTouchdowns', 'ReturnTouchdowns','PointsAllowed','PointsAllowedPerGame','NetPassingYardsAllowed',
     'RushingYardsAllowed','TotalYardsAllowed', 'YardsAgainstPerGame', 'twoPt', 'FantasyPoints','FantasyPointsPerGame']

    #main df to hold all data
    df_load_proj = pd.DataFrame(columns = projection_columns_dbLoad)
    
    # df to hold players that are not in the database for a source yet
    df_missing_players_proj = pd.DataFrame(columns=['date', 'outlet', 'playerId', 'sourceId', 'name'])

    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
    
        directory = r'../data/projection/weekly'
        #looping through every ADP file to aggregate into single df
        for filename in os.listdir(directory):
            f = os.path.join(directory,filename)
            # checking if it is a file
            if os.path.isfile(f):
                temp = pd.read_csv(f, parse_dates=['date'], 
                                   names=['outlet', 'date', 'season', 'week', 'sourceId', 'name', 'shortName', 'pos', 'team', 'GamesPlayed', 
                                          'PassAttempts', 'PassCompletions', 'PassingYards', 'PassingYardsPerGame', 'TouchdownsPasses', 
                                          'InterceptionsThrown', 'PasserRating', 'RushingAttempts', 'RushingYards', 'AverageYardsPerRush', 
                                          'RushingTouchdowns', 'Targets', 'Receptions', 'ReceivingYards', 'YardsPerGame', 'AverageYardsPerReception', 
                                          'ReceivingTouchdowns', 'FumblesLost', 'FieldGoalsMade', 'FieldGoalAttempts', 'LongestFieldGoal', 
                                          'FieldGoals119Yards', 'FieldGoals119YardAttempts', 'FieldGoals2029Yards', 'FieldGoals2029YardAttempts', 
                                          'FieldGoals3039Yards', 'FieldGoals3039YardAttempts', 'FieldGoals4049Yards', 'FieldGoals4049YardAttempts', 
                                          'FieldGoals50Yards', 'FieldGoals50YardsAttempts', 'ExtraPointsMade', 'ExtraPointsAttempted', 'Interceptions', 
                                          'Safeties', 'Sacks', 'Tackles', 'DefensiveFumblesRecovered', 'ForcedFumbles', 'DefensiveTouchdowns', 
                                          'ReturnTouchdowns', 'PointsAllowed', 'PointsAllowedPerGame', 'NetPassingYardsAllowed', 
                                          'RushingYardsAllowed', 'TotalYardsAllowed', 'YardsAgainstPerGame', 'twoPt', 'FantasyPoints', 
                                          'FantasyPointsPerGame'],
                          skiprows=1)

                # creating dicts to convert outlet name/id to db id
                if 'cbs' in f:

                    # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT cbsId, name FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.cbsId.values, index=lookup.name).to_dict()

                    # updating the cbs source data full team name to the abbreviated db name
                    temp['team'] = temp['team'].replace("JAC", "JAX").replace("WAS","WSH")
                    temp.loc[temp['sourceId'].isna(), 'sourceId'] = temp.loc[temp['sourceId'].isna(), 'team'].map(lookup)

                    # cbs source data does not have playerId for the defenses
                    lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

                elif 'espn' in f:
                    # espn source data does not have playerId for the defenses
                     # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT espnId, name FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.espnId.values, index=lookup.name).to_dict()

                    # updating the nfl source data full team name to the abbreviated db name
                    temp.loc[temp['sourceId'].isna(), 'sourceId'] = temp.loc[temp['sourceId'].isna(), 'team'].map(lookup)

                    lookup = pd.read_sql("SELECT espnId, playerId FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()


                elif 'nfl' in f:
                    # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT nflName, name FROM team WHERE nflName IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.name.values, index=lookup.nflName).to_dict()

                    # updating the nfl source data full team name to the abbreviated db name
                    temp.loc[temp['name'].isna(), 'team'] = temp.loc[temp['name'].isna(), 'team'].map(lookup)

                    lookup = pd.read_sql("SELECT playerId, nflId FROM player WHERE nflId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.nflId).to_dict()


                # using the lookup to make the change from outletId to dbId
                temp['playerId'] = temp['sourceId'].map(lookup)

                # creating a df to hold that date for players who are not in the player table for the source
                if temp[pd.isnull(temp['playerId'])].shape[0] > 0:
                    df_missing_players_proj = pd.concat([df_missing_players_proj, 
                                                    temp.loc[pd.isnull(temp['playerId']), ['date', 'outlet', 'playerId', 'sourceId', 'name']]])

                # updating outlet name to db outlet id 
                temp['outlet'] = temp['outlet'].replace(outletLookup)


                temp = temp[projection_columns_dbLoad].replace("-", None).replace("—", None)

                df_load_proj = pd.concat([df_load_proj, temp])

    df_load_proj = df_load_proj.replace(np.nan, None)

    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

In [47]:
if df_missing_players_proj.shape[0] > 0:
    df_missing_players_proj.to_csv('../data/missingPlayersProj.csv')
df_missing_players_proj.shape

(0, 5)

In [48]:
try:
    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
        
        # insert using SQLalchemy
        data = df_load_proj.replace(np.nan, None).values.tolist()
        projMeta = sal.MetaData(engine)
        projTable = sal.Table('projection', projMeta, autoload=True)
        conn.execute(sal.insert(projTable).values(data))

        '''  old connection using mysql.connector
        query_insert = """
            INSERT INTO projection 
            VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(query_insert, df_load_proj.values.tolist())
        conn.commit()
        conn.close()
        '''
    print("success")
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

success


#### INSERT BETTINGPROS LINES

In [49]:
try:
    engine = create_engine(dbConnectionString)
    #cursor = conn.cursor()
    with engine.connect() as conn:
    
        lookup = pd.read_sql("SELECT bpName, teamId FROM team;", conn)
        lookup = pd.Series(lookup.teamId.values, index=lookup.bpName).to_dict()
            
        df_lines['awayTeam'] = df_lines['awayTeam'].map(lookup)
        df_lines['homeTeam'] = df_lines['homeTeam'].map(lookup)

        # insert using SQLalchemy
        data = df_lines.replace(np.nan, None).values.tolist()
        bettingMeta = sal.MetaData(engine)
        bettingTable = sal.Table('betting', bettingMeta, autoload=True)
        conn.execute(sal.insert(bettingTable).values(data))
        
        #### use with the connection from mysql.connector
        '''        
        query_insert = """
            INSERT INTO betting 
                (date, season, week, overUnder, overUnderCost, awayTeamId,
               awaySpread, awayCost, awayMoneyline, homeTeamId, homeSpread,
               homeCost, homeMoneyLine)
            VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(query_insert, df_lines.replace(np.nan, None).values.tolist())
        conn.commit()
        '''
    print("success")
except Exception as ex:
    conn.close()
    print(ex)

success


#### WEEKLY TEAM RECORDS AND

In [50]:
try:
    #conn = mysql.connector.connect(user=dbUser, password=dbPw,host=dbHost,database=dbName)
    #cursor = conn.cursor()
    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
    
        games['homeTeam'] = games['homeTeam'].map(teamLookup)
        games['awayTeam'] = games['awayTeam'].map(teamLookup)
        games['winningTeam'] = games['winningTeam'].map(teamLookup)
        records['team'] = records['team'].map(teamLookup)

        # insert using SQLalchemy
        data = games.replace(np.nan, None).values.tolist()
        gamesMeta = sal.MetaData(engine)
        gamesTable = sal.Table('weeklyScore', gamesMeta, autoload=True)
        conn.execute(sal.insert(gamesTable).values(data))

        # insert using SQLalchemy
        data = records.replace(np.nan, None).values.tolist()
        recordsMeta = sal.MetaData(engine)
        recordsTable = sal.Table('weeklyRecord', recordsMeta, autoload=True)
        conn.execute(sal.insert(recordsTable).values(data))

    '''    THIS WAS USED with the mysql.connector connection
        query_insert1 = """
            INSERT INTO weeklyRecord
                (season, week, teamId, wins, losses, ties)
            VALUES 
                (%s, %s, %s, %s, %s, %s)
        """

        query_insert2 = """
            INSERT INTO weeklyScore 
                (season,week,homeTeamId,homeQ1Pts,homeQ2Pts,homeQ3Pts,homeQ4Pts,
                homeTotalPts,awayTeamId,awayQ1Pts,awayQ2Pts,awayQ3Pts,awayQ4Pts,
                awayTotalPts,homeWinner,awayWinner,tie,winningTeamId,gameId)
            VALUES 
                (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s)
        """

        cursor.executemany(query_insert1, records.replace(np.nan, None).values.tolist())
        cursor.executemany(query_insert2, games.replace(np.nan, None).values.tolist())
        conn.commit()    
        conn.close()
        '''
    print("success")
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

success


# LOADING OLDER DATA

#### INSERTING PLAYER AND TEAM NAMES/ID

In [328]:
try:
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    query_insert = """
        INSERT INTO team 
            (name, espnId, espnName, nflId, nflName, fpId, fpName, cbsId, cbsName, bpId, bpName)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.executemany(query_insert, teams.values.tolist())
    conn.commit()
    conn.close()
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

In [328]:
#players = pd.read_excel("Data/names_backup.csv", sheet_name='names').replace(np.nan, None)#, index_col='playerId').replace(np.nan, None)
#updates, adds, delete, Sheet2
players = pd.read_excel("Data/adds.xlsx", sheet_name='Sheet1')
players = players.replace(np.nan, None).replace(0,None)
players.head(2)

,mflId,sportradarId,gsisId,pffId,sleeperId,yahooId,fleaflickerId,rotowireId,rotoworldId,ktcId,pfrId,cfbrefId,statsId,statsglobalId,fantasydataId,swishId,id
0,15833,9c32f8cb-807a-4864-bc0b-b846d279606d,None,None,8290,None,None,15926,None,None,None,None,33959,None,None,1164968,1520


In [329]:
try:
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    query_insert = """
        INSERT INTO player 
            (posId, teamId, name, espnId, espnName, bpId, bpName, nflId, nflName, fpId, fpName, cbsId, cbsName)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
#    query_insert = """
#        INSERT INTO player 
#            (name, posId, teamId, mflId, sportradarId, fpId,
#            gsisId, pffId, sleeperId, nflId, espnId, 
#            yahooId,fleaflickerId, cbsId, rotowireId, rotoworldId, ktcId, 
#            pfrId, cfbrefId, statsId, statsglobalId, fantasydataId, swishId)
#        VALUES 
#            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#    """
    
    query_update = """
        REPLACE INTO player 
            (playerId, posId, teamId, name, espnId, espnName, bpId, bpName, nflId, nflName, fpId, fpName, cbsId, cbsName)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    query_update1 = """
        UPDATE player 
        SET 
            mflId = %s,
            sportradarId = %s, 
            gsisId = %s, 
            pffId = %s, 
            sleeperId = %s, 
            yahooId = %s, 
            fleaflickerId = %s, 
            rotowireId = %s, 
            rotoworldId = %s, 
            ktcId = %s, 
            pfrId = %s, 
            cfbrefId = %s, 
            statsId = %s, 
            statsglobalId = %s, 
            fantasydataId = %s, 
            swishId = %s
        WHERE playerId = %s
    """
    
    query_insert1 = """
        INSERT INTO player 
            (posId, teamId, cbsId, cbsName, name)
        VALUES 
            (%s, %s, %s, %s, %s)
    """
    
    query_delete = '''
        DELETE FROM player
        WHERE playerId = %s
    '''
    
    query_update = '''
        UPDATE projection SET playerID = %s WHERE playerId = %s
    '''
    
    cursor.executemany(query_update1, players.values.tolist())
    conn.commit()
    conn.close()

except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

#### LOAD ADP

In [867]:
try:
    engine = create_engine(dbConnectionString)
    conn = engine.connect()
    
    # columns for db table adp
    adp_columns = ['outlet', 'date', 'playerId', 'adp','high','low']

    #main df to hold all data
    df_load_adp = pd.DataFrame(columns = adp_columns)

    directory = r"data\adp\2022"
    #looping through every ADP file to aggregate into single df
    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        # checking if it is a file
        if os.path.isfile(f):
            temp = pd.read_excel(f, parse_dates=['date'])

            # creating dicts to convert outlet name/id to db id
            if 'cbs' in f:

                # creating cbs ID to db ID map
                lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

            elif 'espn' in f:
                # espn source data does not have playerId for the defenses
                # converting full team name to db table 'TEAM'.name
                lookup = pd.read_sql("SELECT espnId, name FROM player WHERE espnId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.espnId.values, index=lookup.name).to_dict()

                # updating the player table team name to player table Id
                temp['team'] = temp['team'].str.upper()
                temp.loc[temp['playerId'].isna(), 'playerId'] = temp.loc[temp['playerId'].isna(), 'team'].map(lookup)

                lookup = pd.read_sql("SELECT espnId, playerId FROM player WHERE espnId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()


            elif 'fp' in f:

                # updating the fantasy pro source data id to db id
                temp = temp.loc[pd.notnull(temp['adp'])]

                lookup = pd.read_sql("SELECT playerId, fpId FROM player WHERE fpId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.playerId.values, index=lookup.fpId).to_dict()



            # using the lookup to make the change from outletId to dbId
            temp['playerId'] = temp['playerId'].map(lookup)

            # updating outlet name to db outlet id 
            temp['outlet'] = temp['outlet'].replace(outletLookup)


            temp = temp[adp_columns]
            # adding outlet dataframe to the upload dataframe
            df_load_adp = pd.concat([df_load_adp, temp])

    df_load_adp = df_load_adp.replace(np.nan, None)
    conn.close()
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

c:\users\jrbrz\.virtualenvs\data-adbtfuvq\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\users\jrbrz\.virtualenvs\data-adbtfuvq\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\users\jrbrz\.virtualenvs\data-adbtfuvq\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\users\jrbrz\.virtualenvs\data-adbtfuvq\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQL

In [868]:
try:
    #engine = create_engine(dbConnectionString)
    #conn = engine.connect()
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    query_insert = """
        INSERT INTO adp 

        VALUES 
            (%s, %s, %s, %s, %s, %s)
    """

    cursor.executemany(query_insert, df_load_adp.values.tolist())
    conn.commit()
    conn.close()
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

# <<< SCRATCH >>>

In [97]:
engine = create_engine(dbConnectionString)
with engine.connect() as conn:
    lookup = pd.read_sql("SELECT playerId, nflId FROM player WHERE name = 'Travis Kelce';", conn)
    #lookup = pd.Series(lookup.playerId.values, index=lookup.nflId).to_dict()
nf = pd.read_csv(r"C:\Users\jrbrz\Desktop\projects\projects\ffDraft\data\ranking\weekly\nfl_rank_2022-06_2022-10-13.csv")   

In [680]:
conn.close()

In [76]:
dfs = [df_cbs_proj, df_cbs_ranking, df_cbs_adp, 
       df_fp_ranking, df_fp_adp, 
       df_espn_proj, df_espn_ranking, df_espn_adp,
       df_nfl_proj, df_nfl_ranking,
       df_overUnders]

In [ ]:
service = Service(r"..\browsers\geckodriver.exe")
driver = webdriver.Firefox(service=service)

driver.get("https://www.espn.com/fantasy/football/story/_/id/33898295/fantasy-football-idp-rankings-2022-top-50-defensive-linemen-linebackers-defensive-backs")
time.sleep(10)

# grabs the entire pages html
html = driver.execute_script("return document.body.innerHTML")
soup = bs(html)
driver.close()

In [159]:
# structure to create json file 
c = {}
c = json.dumps(c)
with open('config.txt', 'w') as f:
    f.write(c)